### Code Abstracts

In [1]:
import logging
import json
import sys
import traceback
import pandas as pd
from tqdm import tqdm

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('abstract_scoring_debug.log', mode='w'),
        logging.StreamHandler(sys.stdout)
    ]
)

In [2]:
with open("openai_key.txt", 'r') as f:
    api_key = f.read().strip()

client = OpenAI(api_key=api_key)

In [ ]:
def create_prompt(abstract):
    """
    Safely create prompt template with given abstract
    """
    return f"""
You are coding abstracts on three dimensions. Provide scores between 0 and 1 for each dimension:
1. reproducible: Could we reproduce the results reported in the paper with the provided code/data?
2. results_robust: Are the results robust upon independent verification?
3. data_code_complete: Is the provided data and code complete enough for independent replication?
Return ONLY a valid JSON in this exact format:
{{"reproduce": 0.9, "robust": 0.7, "data_complete": 1.0}}
Abstract:
{abstract}

IMPORTANT: Return ONLY the JSON, nothing else.
"""

def get_scores(abstract, client):
    """
    Get reproducibility scores for an abstract with comprehensive error handling
    """
    try:
        # Log the full abstract being processed
        logging.info(f"Processing abstract: {abstract[:200]}...")
        
        # Create prompt using safe method
        prompt = create_prompt(abstract)
        
        # Make API call
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You're a helpful assistant tasked with coding abstracts based on reproducibility and robustness."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=150,
        )
        
        # Extract response text
        response_text = response.choices[0].message.content.strip()
        
        # Extensive logging of response
        logging.debug(f"Raw response (type: {type(response_text)}): {repr(response_text)}")
        logging.debug(f"Response length: {len(response_text)}")
        
        # Parse JSON
        try:
            scores = json.loads(response_text)
            
            # Validate keys
            if all(key in scores for key in ["reproduce", "robust", "data_complete"]):
                return scores
            else:
                logging.error(f"Missing keys in parsed JSON: {scores}")
                return {"reproduce": None, "robust": None, "data_complete": None}
        
        except json.JSONDecodeError as e:
            logging.error(f"JSON parsing error: {e}")
            logging.error(f"Problematic response: {repr(response_text)}")
            return {"reproduce": None, "robust": None, "data_complete": None}
    
    except Exception as e:
        # Log full stack trace for debugging
        logging.error(f"Unexpected error processing abstract: {abstract[:200]}")
        logging.error(traceback.format_exc())
        return {"reproduce": None, "robust": None, "data_complete": None}

def process_abstracts(df, client):
    """
    Process all abstracts in the DataFrame
    """
    # Process each abstract using tqdm for progress indication
    results = []
    for abstract in tqdm(df['abstract'], desc='Coding abstracts'):
        scores = get_scores(abstract, client)
        results.append(scores)
    
    # Merge results back into the DataFrame
    df_results = pd.DataFrame(results)
    df_final = pd.concat([df, df_results], axis=1)
    
    return df_final

def main():
    try:
        # Load data
        df = pd.read_csv("abstracts.csv")
        
        # TODO: Initialize your client here
        # client = OpenAI(api_key=your_api_key)
        
        # Process abstracts and save results
        df_final = process_abstracts(df, client)
        df_final.to_csv("abstracts_coded.csv", index=False)
        
        logging.info("Processing complete. Results saved to abstracts_coded.csv")
    
    except Exception as e:
        logging.error("Critical error in main execution:")
        logging.error(traceback.format_exc())

if __name__ == "__main__":
    main()

Coding abstracts:   0%|                                 | 0/191 [00:00<?, ?it/s]2025-03-04 18:08:50,240 - INFO - Processing abstract: The long-recognized spurious regressions problem can lead to mistaken inference in panel instrumental variables (IV) estimation. Spurious correlations arising from correlated cycles in finite time hor...
2025-03-04 18:08:50,835 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Coding abstracts:   1%|▏                        | 1/191 [00:00<01:54,  1.65it/s]2025-03-04 18:08:50,846 - INFO - Processing abstract: Increasing evidence suggests that the reproducibility and replicability of scientific findings is threatened by researchers employing questionable research practices (QRP) in order to achieve publisha...
2025-03-04 18:08:51,373 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Coding abstracts:   1%|▎                        | 2/191 [00:01<01:46,  1.77it/s]2025-03-04 18:08:5